[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YOUR_USERNAME/YOUR_REPO_NAME/blob/main/notebooks/visualize_graph.ipynb)

# AI Supply Chain Ontology — Graph Visualizer

This notebook visualizes the ontology graph defined in `ontology.json`.

Behavior:
- Downloads `ontology.json` from your GitHub repo into Colab.
- Builds a directed graph using NetworkX.
- Visualizes selected layers.

Edit the `selected_layers` list below to control which layers are shown.


In [ ]:
# Install and import dependencies
!pip install networkx

import networkx as nx
import matplotlib.pyplot as plt
import os
import json
import urllib.request

In [ ]:
# === DOWNLOAD ontology.json FROM GITHUB ===

# Replace YOUR_USERNAME and YOUR_REPO_NAME with your actual GitHub username and repo name.
RAW_URL = "https://raw.githubusercontent.com/YOUR_USERNAME/YOUR_REPO_NAME/main/ontology.json"
DOWNLOAD_PATH = "/content/ontology.json"

if not os.path.exists(DOWNLOAD_PATH):
    print("Downloading ontology.json from GitHub...")
    urllib.request.urlretrieve(RAW_URL, DOWNLOAD_PATH)
else:
    print("ontology.json already present at", DOWNLOAD_PATH)

print("Ontology path:", DOWNLOAD_PATH)

with open(DOWNLOAD_PATH, 'r') as f:
    data = json.load(f)

nodes = data["nodes"]
edges = data["edges"]

print(f"Loaded {len(nodes)} nodes and {len(edges)} edges.")

In [ ]:
# === CONFIG — SELECT LAYERS TO VISUALIZE ===
# Example:
#   [2]       → only Layer 2
#   [1,2,3]   → Layers 1–3
#   []        → ALL layers

selected_layers = [2]  # <-- edit this line as needed

NODE_SIZE = 900

# === BUILD GRAPH ===
G = nx.DiGraph()

for n in nodes:
    G.add_node(n["id"], **n)

for e in edges:
    G.add_edge(e["source"], e["target"], **e)

# === FILTER BY LAYERS ===
if selected_layers:
    H = nx.DiGraph()
    for node_id, attrs in G.nodes(data=True):
        if attrs.get("layer") in selected_layers:
            H.add_node(node_id, **attrs)
    for u, v, attrs in G.edges(data=True):
        if u in H.nodes and v in H.nodes:
            H.add_edge(u, v, **attrs)
else:
    H = G.copy()

if H.number_of_nodes() == 0:
    print("No nodes to display for selected_layers =", selected_layers)
else:
    # Color by layer
    cmap = plt.cm.get_cmap('tab10')
    def color_for_layer(layer):
        if layer is None:
            return (0.5, 0.5, 0.5, 1.0)
        return cmap((layer - 1) % 10)

    node_colors = [color_for_layer(d.get("layer")) for _, d in H.nodes(data=True)]
    labels = {n: d.get("name", n) for n, d in H.nodes(data=True)}

    pos = nx.spring_layout(H, seed=42, k=1.1)

    plt.figure(figsize=(14, 9))

    # Draw nodes
    nx.draw_networkx_nodes(H, pos,
                           node_color=node_colors,
                           node_size=NODE_SIZE,
                           alpha=0.9)

    # Separate edges by type
    dep_edges = [(u, v) for u, v, d in H.edges(data=True) if d.get("type") == "dependency"]
    comp_edges = [(u, v) for u, v, d in H.edges(data=True) if d.get("type") == "competition"]
    lev_edges  = [(u, v) for u, v, d in H.edges(data=True) if d.get("type") == "leverage"]

    nx.draw_networkx_edges(H, pos, edgelist=dep_edges, arrows=True, arrowstyle='-|>', alpha=0.7)
    nx.draw_networkx_edges(H, pos, edgelist=lev_edges, arrows=True, arrowstyle='-|>', style='dashed', alpha=0.7)
    nx.draw_networkx_edges(H, pos, edgelist=comp_edges, arrows=False, style='dotted', alpha=0.7)

    nx.draw_networkx_labels(H, pos, labels, font_size=9)

    title = "AI Supply Chain Ontology — Layers " + (str(selected_layers) if selected_layers else "ALL")
    plt.title(title)
    plt.axis('off')
    plt.show()

    present_layers = sorted({d.get("layer") for _, d in H.nodes(data=True)})
    print("Layers present in this view:", present_layers)
